## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate  source  \
0  2025-10-21-23-51-10 +0000     nyt   
1  2025-10-21-23-47-13 +0000  nypost   
2  2025-10-21-23-43-41 +0000     bbc   
3  2025-10-21-23-40-00 +0000     wsj   
4  2025-10-21-23-38-00 +0000     wsj   

                                               title  \
0  In Trump’s Washington, Hate Is Not a Deal Breaker   
1  Texas parents accused of burying autistic son ...   
2  The £5.30 orange juice that tells the story of...   
3  Texas Instruments Warns of Slower Semiconducto...   
4  Paul Ingrassia withdrew Tuesday from a Senate ...   

                                                link  
0  https://www.nytimes.com/2025/10/21/us/politics...  
1  https://nypost.com/2025/10/21/us-news/texas-pa...  
2  https://www.bbc.com/news/articles/c397n3jl3z8o...  
3  https://www.wsj.com/business/earnings/texas-in...  
4  https://www.wsj.com/politics/policy/paul-ingra...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2320/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
0    trump     55
42   house     17
293     up     14
41   white     14
311  hamas     13

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
239  2025-10-21-11-43-48 +0000     cbc   
130  2025-10-21-18-44-04 +0000     bbc   
322  2025-10-21-02-48-43 +0000     bbc   
142  2025-10-21-17-55-57 +0000  nypost   
14   2025-10-21-23-22-25 +0000     nyt   

                                                 title  \
239  White House demolition work begins as Trump pu...   
130  From bowling alley to ballroom - Trump isn't t...   
322  White House begins demolishing part of East Wi...   
142  Hillary mocked after blasting Trump’s White Ho...   
14   As Trump Moves Forward With White House Ballro...   

                                                  link  score  
239  https://www.cbc.ca/news/world/us-white-house-d...    110  
130  https://www.bbc.com/news/articles/cr7m2v80dy4o...    109  
322  https://www.bbc.com/news/articles/ced6np51532o...    108  
142  https://nypost.com/2025/10/21/us-news/hillary-...    108  
14   https://www.nytimes.com/2025/10/21/arts/design...    103

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate   source  \
239    110  2025-10-21-11-43-48 +0000      cbc   
36      74  2025-10-21-22-12-51 +0000      nyt   
317     54  2025-10-21-04-53-16 +0000     wapo   
1       48  2025-10-21-23-47-13 +0000   nypost   
220     41  2025-10-21-13-25-26 +0000   nypost   
143     40  2025-10-21-17-44-24 +0000   nypost   
215     39  2025-10-21-13-56-46 +0000   nypost   
38      39  2025-10-21-22-11-36 +0000      nyt   
275     35  2025-10-21-09-30-00 +0000      wsj   
13      35  2025-10-21-23-22-38 +0000  latimes   

                                                 title  \
239  White House demolition work begins as Trump pu...   
36   Vance Lands in Israel as U.S. Tries to Shore U...   
317  Japan elects first female leader, a hawk who p...   
1    Texas parents accused of burying autistic son ...   
220  Former French President Sarkozy begins 5-year ...   
143  Hamas released bodies of two more dead hostage...   
215  Warner Bros. Discovery says it’s open to a sal...   
38   Paris Tour Guide Saw Stolen Louvre Jewels Just...   
275  NBCUniversal Made a $27 Billion Bet on the NBA...   
13   California cop, 25, is killed as she stops to ...   

                                                  link  
239  https://www.cbc.ca/news/world/us-white-house-d...  
36   https://www.nytimes.com/2025/10/21/world/middl...  
317  https://www.washingtonpost.com/world/2025/10/2...  
1    https://nypost.com/2025/10/21/us-news/texas-pa...  
220  https://nypost.com/2025/10/21/world-news/forme...  
143  https://nypost.com/2025/10/21/world-news/hamas...  
215  https://nypost.com/2025/10/21/business/warner-...  
38   https://www.nytimes.com/2025/10/21/style/louvr...  
275  https://www.wsj.com/business/media/nbcuniversa...  
13   https://www.latimes.com/california/story/2025-...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
